# 5_analyzing_data.py

This notebook was automatically converted from a Python script.

# Pandas ile Veri Analizi


Pandas, veri analizi için güçlü ve esnek araçlar sunar. Bu bölümde, Pandas kullanarak veri analizi nasıl yapılır öğreneceğiz.


## Veri Analizi Nedir?


Veri analizi, ham verilerden anlamlı bilgiler çıkarmayı, sonuçları yorumlamayı ve karar vermeyi desteklemek amacıyla yapılan bir süreçtir. Bu süreç genellikle şu adımları içerir:


1. Veriyi anlama ve keşfetme


2. Veriyi temizleme ve hazırlama


3. Veriyi dönüştürme ve zenginleştirme


4. İstatistiksel analiz ve modelleme


5. Sonuçları yorumlama ve görselleştirme


Şimdi bu adımları Pandas kullanarak nasıl gerçekleştirebileceğimize bakalım.


Gerekli kütüphaneleri import edelim


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



Grafikleri not defterinde görüntülemek için (Jupyter Notebook'ta çalıştırılırsa gerekli)


%matplotlib inline


Rasgele verilerin tutarlı olması için seed ayarlayalım


In [ ]:
np.random.seed(42)



## Örnek Veri Seti Oluşturma


Bir e-ticaret sitesinin satış verilerini simüle eden bir veri seti oluşturalım.


Tarih aralığı oluşturma


In [ ]:
tarihler = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')



Kategoriler


In [ ]:
kategoriler = ['Elektronik', 'Giyim', 'Kitap', 'Ev', 'Spor']



Satış verileri oluşturma


In [ ]:
n_rows = len(tarihler)
data = {
    'Tarih': tarihler,
    'Kategori': np.random.choice(kategoriler, size=n_rows),
    'Satış': np.random.randint(100, 10000, size=n_rows),
    'Miktar': np.random.randint(1, 100, size=n_rows),
    'İade': np.random.randint(0, 10, size=n_rows),
    'Müşteri_ID': np.random.randint(1000, 5000, size=n_rows),
    'Ödeme_Tipi': np.random.choice(['Kredi Kartı', 'Havale', 'Kapıda Ödeme'], size=n_rows)
}



DataFrame oluşturma


In [ ]:
df = pd.DataFrame(data)



Verilere biraz gürültü ve eksik değerler ekleyelim


In [ ]:
df.loc[np.random.choice(df.index, 50), 'Satış'] = np.nan
df.loc[np.random.choice(df.index, 30), 'Miktar'] = np.nan



Veri setini görelim


In [ ]:
print("Örnek veri seti:")
print(df.head())



## 1. Veriyi Anlama ve Keşfetme


Veri analizi sürecinin ilk adımı, elimizdeki veriyi anlamak ve keşfetmektir. Pandas, bunun için çeşitli araçlar sunar.


In [ ]:

print("\n--- Veriyi Anlama ve Keşfetme ---")



Verinin yapısını inceleme


In [ ]:
print("\nVeri seti boyutu:", df.shape)
print("\nVeri seti sütunları:", df.columns.tolist())
print("\nVeri seti bilgileri:")
df.info()



İstatistiksel özet


In [ ]:
print("\nİstatistiksel özet:")
print(df.describe())



Kategorik veri özetleri


In [ ]:
print("\nKategori dağılımı:")
print(df['Kategori'].value_counts())

print("\nÖdeme tipi dağılımı:")
print(df['Ödeme_Tipi'].value_counts(normalize=True))  # Yüzde olarak



Zaman serisi verilerini anlama


In [ ]:
print("\nZaman bazlı özet:")
print(df.set_index('Tarih').resample('M')['Satış'].sum())



## 2. Veriyi Temizleme ve Hazırlama


Veri analizi sürecinde en çok zaman alan adımlardan biri veri temizleme ve hazırlamadır. Bu adım, eksik değerleri ele alma, aykırı değerleri tespit etme ve gereksiz sütunları kaldırma gibi işlemleri içerir.


In [ ]:

print("\n--- Veriyi Temizleme ve Hazırlama ---")



Eksik değerleri tespit etme


In [ ]:
print("\nEksik değerler:")
print(df.isnull().sum())



Eksik değerleri doldurma


In [ ]:
df_clean = df.copy()
df_clean['Satış'] = df_clean['Satış'].fillna(df_clean['Satış'].mean())
df_clean['Miktar'] = df_clean['Miktar'].fillna(df_clean['Miktar'].median())

print("\nEksik değerler doldurulduktan sonra:")
print(df_clean.isnull().sum())



Aykırı değerleri tespit etme ve işleme


Satış sütununun kutu grafiğini çizerek aykırı değerleri görebiliriz


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_clean['Satış'])
plt.title('Satış Değerleri Dağılımı')
plt.savefig('satis_boxplot.png')  # Grafiği kaydetme
plt.close()



Z-skoru ile aykırı değerleri tespit etme


In [ ]:
def detect_outliers(df, column, threshold=3):
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    return df[z_scores > threshold].index

outliers_index = detect_outliers(df_clean, 'Satış')
print(f"\nAykırı değerlerin sayısı: {len(outliers_index)}")



Aykırı değerleri işleme (IQR yöntemi)


In [ ]:
Q1 = df_clean['Satış'].quantile(0.25)
Q3 = df_clean['Satış'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR



Aykırı değerleri kırpma (capping)


In [ ]:
df_clean['Satış_Capped'] = df_clean['Satış'].clip(lower_bound, upper_bound)

print("\nAykırı değerler işlendikten sonra istatistiksel özet:")
print(df_clean[['Satış', 'Satış_Capped']].describe())



Veri tiplerini doğru şekilde ayarlama


In [ ]:
df_clean['Tarih'] = pd.to_datetime(df_clean['Tarih'])
df_clean['Yıl'] = df_clean['Tarih'].dt.year
df_clean['Ay'] = df_clean['Tarih'].dt.month
df_clean['Gün'] = df_clean['Tarih'].dt.day
df_clean['Haftanın_Günü'] = df_clean['Tarih'].dt.day_name()

print("\nTarih bileşenleri eklendikten sonra:")
print(df_clean.head())



## 3. Veriyi Dönüştürme ve Zenginleştirme


Veri temizleme işleminin ardından, veriyi analiz için daha uygun hale getirmek amacıyla dönüştürme ve zenginleştirme işlemleri yapılır.


In [ ]:

print("\n--- Veriyi Dönüştürme ve Zenginleştirme ---")



Özellik mühendisliği: Yeni özellikler oluşturma


In [ ]:
df_clean['Birim_Fiyat'] = df_clean['Satış'] / df_clean['Miktar']
df_clean['İade_Oranı'] = df_clean['İade'] / df_clean['Miktar']
df_clean['İade_Var_Mı'] = df_clean['İade'] > 0

print("\nYeni özellikler eklendikten sonra:")
print(df_clean.head())



Kategorik verileri sayısal değerlere dönüştürme (one-hot encoding)


In [ ]:
df_clean_encoded = pd.get_dummies(df_clean, columns=['Kategori', 'Ödeme_Tipi'], drop_first=True)
print("\nOne-hot encoding sonrası sütunlar:")
print(df_clean_encoded.columns.tolist())



Veriyi normalize etme (ölçeklendirme)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numeric_cols = ['Satış', 'Miktar', 'Birim_Fiyat']
df_clean[numeric_cols + ['_Normalized']] = scaler.fit_transform(df_clean[numeric_cols])

print("\nNormalizasyon sonrası:")
print(df_clean[numeric_cols + ['_Normalized']].head())



## 4. İstatistiksel Analiz


Temizlenmiş ve hazırlanmış veriye çeşitli istatistiksel analizler uygulayabiliriz.


In [ ]:

print("\n--- İstatistiksel Analiz ---")



Korelasyon analizi


In [ ]:
corr = df_clean[['Satış', 'Miktar', 'İade', 'Birim_Fiyat', 'İade_Oranı']].corr()
print("\nKorelasyon matrisi:")
print(corr)



Korelasyon matrisini görselleştirme


In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Korelasyon Matrisi')
plt.savefig('korelasyon_matrisi.png')
plt.close()



Kategori bazında analiz


In [ ]:
kategori_analiz = df_clean.groupby('Kategori').agg({
    'Satış': ['sum', 'mean', 'count'],
    'Miktar': ['sum', 'mean'],
    'İade': ['sum', 'mean'],
    'İade_Oranı': 'mean'
})

print("\nKategori bazında analiz:")
print(kategori_analiz)



Zaman serisi analizi


In [ ]:
aylık_satış = df_clean.set_index('Tarih').resample('M')['Satış'].sum()
print("\nAylık toplam satış:")
print(aylık_satış)



Ödeme tipine göre satış analizi


In [ ]:
odeme_analiz = df_clean.groupby('Ödeme_Tipi').agg({
    'Satış': ['sum', 'mean', 'count'],
    'Müşteri_ID': pd.Series.nunique
}).sort_values(('Satış', 'sum'), ascending=False)

print("\nÖdeme tipine göre satış analizi:")
print(odeme_analiz)



Haftalık satış trendi


In [ ]:
haftalık_satış = df_clean.set_index('Tarih').resample('W-MON')['Satış'].sum()

plt.figure(figsize=(14, 7))
haftalık_satış.plot()
plt.title('Haftalık Satış Trendi')
plt.xlabel('Tarih')
plt.ylabel('Toplam Satış')
plt.grid(True)
plt.savefig('haftalik_satis_trendi.png')
plt.close()



## 5. İleri Analiz Yöntemleri


Pandas ile daha karmaşık analizler de yapabiliriz.


In [ ]:

print("\n--- İleri Analiz Yöntemleri ---")



Pivot tablolar


In [ ]:
pivot = pd.pivot_table(df_clean, 
                     values='Satış',
                     index='Ay',
                     columns='Kategori',
                     aggfunc='sum',
                     fill_value=0)

print("\nAylara ve kategorilere göre satış pivot tablosu:")
print(pivot)



Pivot tabloyu görselleştirme


In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(pivot, annot=True, fmt='.0f', cmap='YlGnBu')
plt.title('Aylara ve Kategorilere Göre Satış')
plt.savefig('pivot_tablo.png')
plt.close()



Kategori ve ödeme tipine göre çapraz tablo


In [ ]:
cross_tab = pd.crosstab(df_clean['Kategori'], df_clean['Ödeme_Tipi'], 
                        values=df_clean['Satış'], aggfunc='sum', normalize='index')

print("\nKategori ve ödeme tipine göre satış çapraz tablosu (yüzde):")
print(cross_tab)



Satışların en yüksek olduğu müşteriler


In [ ]:
top_customers = df_clean.groupby('Müşteri_ID').agg({
    'Satış': 'sum',
    'Miktar': 'sum',
    'İade': 'sum'
}).sort_values('Satış', ascending=False).head(10)

print("\nEn çok satış yapılan 10 müşteri:")
print(top_customers)



Zamana göre büyüme analizi


Aydan aya büyüme oranını hesaplama


In [ ]:
aylık_satış_df = aylık_satış.reset_index()
aylık_satış_df.columns = ['Tarih', 'Satış']
aylık_satış_df['Önceki_Ay_Satış'] = aylık_satış_df['Satış'].shift(1)
aylık_satış_df['Büyüme_Oranı'] = (aylık_satış_df['Satış'] - aylık_satış_df['Önceki_Ay_Satış']) / aylık_satış_df['Önceki_Ay_Satış'] * 100

print("\nAydan aya büyüme oranları:")
print(aylık_satış_df)



## 6. Veri Görselleştirme


Pandas, veri görselleştirme için matplotlib ve seaborn kütüphaneleri ile entegre çalışır.


In [ ]:

print("\n--- Veri Görselleştirme ---")



Kategorilere göre toplam satış grafiği


In [ ]:
plt.figure(figsize=(12, 6))
kategori_satış = df_clean.groupby('Kategori')['Satış'].sum().sort_values(ascending=False)
kategori_satış.plot(kind='bar', color='skyblue')
plt.title('Kategorilere Göre Toplam Satış')
plt.xlabel('Kategori')
plt.ylabel('Toplam Satış')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('kategori_satis.png')
plt.close()



Aylara göre satış trendi


In [ ]:
plt.figure(figsize=(14, 7))
ay_satış = df_clean.groupby('Ay')['Satış'].sum()
ay_satış.plot(kind='line', marker='o', color='green', linewidth=2)
plt.title('Aylara Göre Satış Trendi')
plt.xlabel('Ay')
plt.ylabel('Toplam Satış')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig('ay_satis_trendi.png')
plt.close()



Ödeme tipine göre satış dağılımı


In [ ]:
plt.figure(figsize=(10, 7))
df_clean.groupby('Ödeme_Tipi')['Satış'].sum().plot(kind='pie', autopct='%1.1f%%', startangle=90, explode=[0.05, 0, 0])
plt.title('Ödeme Tipine Göre Satış Dağılımı')
plt.ylabel('')
plt.savefig('odeme_tipi_dagilimi.png')
plt.close()



Miktar ve satış ilişkisi


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='Miktar', y='Satış', hue='Kategori', alpha=0.6)
plt.title('Miktar ve Satış İlişkisi')
plt.savefig('miktar_satis_iliskisi.png')
plt.close()



Kutu grafikleri ile kategorilere göre satış dağılımı


In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='Kategori', y='Satış', data=df_clean)
plt.title('Kategorilere Göre Satış Dağılımı')
plt.savefig('kategori_satis_dagilimi.png')
plt.close()



Haftanın günlerine göre satış yoğunluğu


In [ ]:
plt.figure(figsize=(12, 6))
gun_sırası = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
gun_satış = df_clean.groupby('Haftanın_Günü')['Satış'].mean()
gun_satış = gun_satış.reindex(gun_sırası)
sns.barplot(x=gun_satış.index, y=gun_satış.values, palette='viridis')
plt.title('Haftanın Günlerine Göre Ortalama Satış')
plt.xlabel('Haftanın Günü')
plt.ylabel('Ortalama Satış')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('gun_satis.png')
plt.close()



## 7. Sonuçlar ve Çıkarımlar


Analizlerimizden çıkan bazı önemli bulgular:


In [ ]:

print("\n--- Sonuçlar ve Çıkarımlar ---")



En çok satış yapılan kategori


In [ ]:
print(f"\nEn çok satış yapılan kategori: {kategori_satış.index[0]}, Toplam Satış: {kategori_satış.iloc[0]:.2f}")



En çok satış yapılan ay


In [ ]:
print(f"En çok satış yapılan ay: {ay_satış.idxmax()}, Toplam Satış: {ay_satış.max():.2f}")



En yüksek ortalama satış olan ödeme tipi


In [ ]:
ortalama_odeme = df_clean.groupby('Ödeme_Tipi')['Satış'].mean()
print(f"En yüksek ortalama satış olan ödeme tipi: {ortalama_odeme.idxmax()}, Ortalama Satış: {ortalama_odeme.max():.2f}")



İade oranı en yüksek olan kategori


In [ ]:
iade_oranı = df_clean.groupby('Kategori')['İade_Oranı'].mean()
print(f"İade oranı en yüksek olan kategori: {iade_oranı.idxmax()}, Ortalama İade Oranı: {iade_oranı.max():.2%}")



Satış miktarı ve toplam satış arasındaki korelasyon


In [ ]:
print(f"Satış miktarı ve toplam satış arasındaki korelasyon: {df_clean['Miktar'].corr(df_clean['Satış']):.2f}")



## Özet


Bu bölümde Pandas ile veri analizi sürecini adım adım öğrendik:


1. Veriyi anlama ve keşfetme


2. Veriyi temizleme ve hazırlama


3. Veriyi dönüştürme ve zenginleştirme


4. İstatistiksel analiz ve modelleme


5. Sonuçları yorumlama ve görselleştirme


Pandas, veri analizi sürecinin her adımında kullanışlı araçlar sunar. Bu örnekte gördüğümüz gibi, ham verileri alıp anlamlı sonuçlar ve görselleştirmeler elde edebilir, böylece veri odaklı kararlar verebiliriz.


Veri analizi, özellikle iş dünyasında, pazarlamada, finansta ve bilimsel araştırmalarda kritik bir beceridir. Pandas kullanarak verileri analiz etme yeteneği, veri bilimciler, analistler ve araştırmacılar için vazgeçilmez bir araçtır. 
